In [1]:
import sys

from itertools import chain, combinations, islice
from collections import defaultdict
from optparse import OptionParser
from openpyxl import load_workbook
from operator import mul
from functools import reduce
import pandas as pd
import operator

In [2]:
df = pd.read_csv('siswa.csv')
df.head()

,No,No_Induk,Nama,jen_kel,alamat,Agama,Tgl_Lahir,Tmpt_Lahir,Nama_Wali,Pekerjaan Wali,Pend,Alamat_Wali,Thn_msk,Usia_msk,lulus,UAS,predikat,Pend_SebelumSD
1,2683,Ani Yuanita,P,Tegallayang,Islam,03/11/1993,Bantul,Sukarjo,Petani,SD,Tegallayang,2000,7,2007,17,9,Cukup,TK
2,2686,Burhan Fajar P.,L,Tegallayang,Islam,29/12/1993,Bantul,Hadi Wardoyo,Petani,SMA,Tegallayang,2000,7,2007,16,9,Cukup,TK
3,2688,Heri Budiyanto,L,Tegallayang,Islam,15/02/1994,Bantul,Ribut,Petani,NaN,Tegallayang,2000,6,2007,20,15,Cukup,TK
4,2700,Dimas Satrio W.,L,Tegallayang,Islam,20/04/1994,Bantul,FX Slamet Sudarsono,PNS,PT,Tegallayang,2001,7,2007,18,35,Cukup,TK
5,2701,Nur Hidayat,L,Tegallayang,Islam,12/06/1994,Kuningan,Ofan,Wiraswasta,SLTP,Tegallayang,2001,7,2007,26,1,Baik,TK


In [3]:
df.dtypes.head()

No           int64
No_Induk    object
Nama        object
jen_kel     object
alamat      object
dtype: object

In [4]:
d = df[['Usia_msk','predikat','Pend_SebelumSD']]
d.head()

,Usia_msk,predikat,Pend_SebelumSD
1,2007,Cukup,TK
2,2007,Cukup,TK
3,2007,Cukup,TK
4,2007,Cukup,TK
5,2007,Baik,TK


In [5]:
def subsets(arr):
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])

def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
        
        _itemSet = set()
        localSet = defaultdict(int)

        for item in itemSet:
                for transaction in transactionList:
                        if item.issubset(transaction):
                                freqSet[item] += 1
                                localSet[item] += 1

        for item, count in localSet.items():
                support = float(count)/len(transactionList)

                if support >= minSupport:
                        _itemSet.add(item)

        return _itemSet
def joinSet(itemSet, length):
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])

def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))              # Generate 1-itemSets
    return itemSet, transactionList

def runApriori(data_iter, minSupport, minConfidence):
   
    itemSet, transactionList = getItemSetTransactionList(data_iter)

    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet,
                                        transactionList,
                                        minSupport,
                                        freqSet)

    currentLSet = oneCSet
    k = 2
    while(currentLSet != set([])):
        largeSet[k-1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet,
                                                transactionList,
                                                minSupport,
                                                freqSet)
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
            return float(freqSet[item])/len(transactionList)

    toRetItems = []
    for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item))
                           for item in value])

    toRetRules = []
    for key, value in islice(largeSet.items(), 1, None):
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = getSupport(item)/getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)),
                                           confidence))
    return toRetItems, toRetRules


def printResults(items, rules):
    dSupport = {}
    for item, support in sorted(items, key=operator.itemgetter(1)):
        dSupport[item] = support
        print("item: %s , %.3f" % (str(item), support))

    for rule, confidence in sorted(rules, key=operator.itemgetter(1)):
        pre, post = rule
        tempRule=()
        tempRule+=pre
        tempRule+=post
        supportAimplikasiB = getSupportAimplikasiB(tempRule, dSupport)
        lift = supportAimplikasiB/(dSupport[pre]*dSupport[post])
        print("Rule: %s ==> %s : [confidence is %.3f, lift is %.3f]" % (str(pre), str(post), confidence, lift))

def getSupportAimplikasiB(A, B):
    rule = list(A)
    dict_support = B
    result = 0
    for index_dict_support in dict_support:
        list_index_dict_support = list(index_dict_support)
        check = []
        for data_index in list_index_dict_support:
            if data_index in rule and len(list_index_dict_support) >= len(rule):
                check.append(True)
            else:
                check.append(False)
        if(reduce(mul, check, 1)==True):
#             print(list_index_dict_support)
#             print(dict_support[index_dict_support])
            result = dict_support[index_dict_support]
    
    return result

def dataFromFile(fname):
        file_iter = open(fname, 'rU')
        for line in file_iter:
                line = line.strip().rstrip(',')                         # Remove trailing comma
                record = frozenset(line.split(','))
                yield record

In [6]:
list_data, temp_data = [],[]

for index, data in d.iterrows():
    if (data['Usia_msk']==2007):
        temp_data.append(2007)
    else:
        temp_data.append(2008)
    
    if (data['predikat']=='Baik'):
        temp_data.append('Baik')
    else:
        temp_data.append('Cukup')
        
    if (data['Pend_SebelumSD']=='TK'):
        temp_data.append('TK')
    else:
        temp_data.append('None')
        
    list_data.append(temp_data)
    temp_data = []

In [7]:
minSupport = 0.1
minConfidence = 0.2

items, rules = runApriori(list_data, minSupport, minConfidence)
printResults(items, rules)

item: (2008, 'Baik') , 0.103
item: (2008, 'Cukup') , 0.103
item: (2008, 'TK', 'Cukup') , 0.103
item: (2008, 'TK') , 0.172
item: (2008,) , 0.207
item: ('Cukup', 2007) , 0.345
item: ('TK', 'Cukup', 2007) , 0.345
item: ('Cukup',) , 0.448
item: ('Baik', 2007) , 0.448
item: ('TK', 'Cukup') , 0.448
item: ('Baik', 2007, 'TK') , 0.448
item: ('Baik', 'TK') , 0.517
item: ('Baik',) , 0.552
item: (2007,) , 0.793
item: ('TK', 2007) , 0.793
item: ('TK',) , 0.966
Rule: ('Cukup',) ==> (2008,) : [confidence is 0.231, lift is 1.115]
Rule: ('Cukup',) ==> (2008, 'TK') : [confidence is 0.231, lift is 1.338]
Rule: ('TK', 'Cukup') ==> (2008,) : [confidence is 0.231, lift is 1.115]
Rule: ('TK',) ==> ('Cukup', 2007) : [confidence is 0.357, lift is 1.036]
Rule: (2007,) ==> ('Cukup',) : [confidence is 0.435, lift is 0.970]
Rule: (2007,) ==> ('TK', 'Cukup') : [confidence is 0.435, lift is 0.970]
Rule: ('TK', 2007) ==> ('Cukup',) : [confidence is 0.435, lift is 0.970]
Rule: ('TK',) ==> ('Cukup',) : [confidence is 